## Import and connect Google colab to Google drive

In [1]:
import glob, os
import numpy as np
from numpy import save, load
import pandas as pd
import itertools
import random
#torch

#Google colab tools
from google.colab import files # To handle files and eg export to your browser
import glob # To handle files and eg export to your browser
from google.colab import drive # Mount your Google drive
# matplot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from IPython.display import Image, clear_output  # to display images
np.random.seed()
sns.set(style='white', context='notebook', palette='deep')


In [2]:
# The way to import/export a file with google colab to you google drive

drive.mount('/content/drive/')

IMAGEHUB='/content/drive/My Drive/Robotsports/Robotsports_imagehub'


#Test it
!ls '$IMAGEHUB'

Mounted at /content/drive/
20201014_goalpictures	     Other
20201201_iphone_test_images  Robotsport_imagehub_tools.ipynb
20201202_field_filtered      validation_set


## Translate VOCXML annotation to Yolo format

In [3]:
#
# @author:  Noah van der Meer
# @brief:   VOC XML to YOLO annotation converter
#
# @date:    28-01-2021
#

from typing import List, Tuple, Dict  # for Python type hinting/static typing

import os                       # listing and reading files
import xml.dom.minidom          # parsing XML in DOM

def voc_xml_to_yolo(dom, classes: Dict[str, int]) -> str:
    """ Convert from VOC XML to Yolo annotation format

    Parameters
    ----------
    dom : 
        loaded xml document

    classes : Dict
        class name -> index map


    The YOLO format:
    <class> <x> <y> <width> <height>

    where
    <class>             - integer number of object from 0 to (numClasses - 1)
    <x> <y>             - relative x,y coordinates of _center_ of bounding box
    <width> <height>    - relative bounding box width/height

    More information about the YOLO format:
    https://github.com/AlexeyAB/Yolo_mark/issues/60


    
    Returns
    -------
    str :
        annotations encoded in YOLO format

    """

    output = ""

    annotation = dom.documentElement
    
    # Determine image size
    size = annotation.getElementsByTagName("size")[0]
    imgWidth: int = int(size.getElementsByTagName("width")[0].childNodes[0].data)
    imgHeight: int = int(size.getElementsByTagName("height")[0].childNodes[0].data)

    # Iterate through objects
    objects = annotation.getElementsByTagName("object")
    for obj in objects:
        # Look up class
        name: str = obj.getElementsByTagName("name")[0].childNodes[0].data
        name.replace(" ", "")
        if name in classes:
            classId = classes[name]
        else:
            print("-- voc_xml_to_yolo() error: class '", name, 
                    "' not known! Skipping object")
            continue

        # Interpret bounding box
        bndbox = obj.getElementsByTagName("bndbox")[0]
        xmin: int = int(bndbox.getElementsByTagName("xmin")[0].childNodes[0].data)
        ymin: int = int(bndbox.getElementsByTagName("ymin")[0].childNodes[0].data)
        xmax: int = int(bndbox.getElementsByTagName("xmax")[0].childNodes[0].data)
        ymax: int = int(bndbox.getElementsByTagName("ymax")[0].childNodes[0].data)
        
        xCenter: float = (xmin + xmax) / (2 * imgWidth)
        yCenter: float = (ymin + ymax) / (2 * imgHeight)
        bndWidth: float = (xmax - xmin) / imgWidth
        bndHeight: float = (ymax - ymin) / imgHeight

        output = output + str(classId) + " " + str(xCenter) + " " + str(yCenter) + " " + str(bndWidth) + " " + str(bndHeight) + "\n"

    return output


def voc_xml_directory_to_yolo(sourceDirectory: str, targetDirectory: str, classes: Dict[str, int]):
    """ Convert from VOC XML annotations to Yolo annotation format

    Parameters
    ----------
    sourceDirectory : str
        path to source directory containing annotations in VOC XML format. This method
        will attempt to load any file with the *.xml extension, and it ignores other files

    targetDirectory : str
        target directory for writing the annotation files in Yolo format

    classes : Dict
        class name -> index map

    """

    files: List[str] = os.listdir(sourceDirectory)
    print("voc_xml_directory_to_yolo() : found", len(files), "files in directory", sourceDirectory)

    for filename in files:

        # check file extension
        if len(filename) < 3:
            continue
        if filename[-3:] != "xml":
            continue
        
        targetFilename = filename[0:-3] + "txt"
        print("voc_xml_directory_to_yolo() :", filename, "->", targetFilename)

        dom = xml.dom.minidom.parse(sourceDirectory + "/" + filename)
        yl: str = voc_xml_to_yolo(dom, classes)

        # write to file
        fo = open(targetDirectory + "/" + targetFilename, "w")
        fo.write(yl)
        fo.close()




In [6]:
classes = {"ball": 0, "robot": 1, "human": 2, "goalpost": 3, "goal": 4}

DIR = '20201201_iphone_test_images'  # @param

voc_xml_directory_to_yolo(IMAGEHUB+'/' + DIR + '/VOCXML', 
                        IMAGEHUB+'/' + DIR +'/YOLO',
                        classes)

voc_xml_directory_to_yolo() : found 10 files in directory /content/drive/My Drive/Robotsports/Robotsports_imagehub/20201201_iphone_test_images/VOCXML
voc_xml_directory_to_yolo() : IMG_2865.xml -> IMG_2865.txt
voc_xml_directory_to_yolo() : IMG_2866.xml -> IMG_2866.txt
voc_xml_directory_to_yolo() : IMG_2867.xml -> IMG_2867.txt
voc_xml_directory_to_yolo() : IMG_2868.xml -> IMG_2868.txt
voc_xml_directory_to_yolo() : IMG_2869.xml -> IMG_2869.txt
voc_xml_directory_to_yolo() : IMG_2870.xml -> IMG_2870.txt
voc_xml_directory_to_yolo() : IMG_2871.xml -> IMG_2871.txt
voc_xml_directory_to_yolo() : IMG_2872.xml -> IMG_2872.txt
voc_xml_directory_to_yolo() : IMG_2874.xml -> IMG_2874.txt
voc_xml_directory_to_yolo() : IMG_2873.xml -> IMG_2873.txt


## Convert PGN to JPG images

In [ ]:
#
# @author:  Noah van der Meer
# @brief:   PNG to JPG image conversion through OpenCV
#
# @date:    14-02-2021
#

from typing import List, Tuple, Dict  # for Python type hinting/static typing

import cv2                            # PNG and JPEG encoding

import os                             # listing files

def png_directory_to_jpg(sourceDirectory: str, targetDirectory: str):
    """ Convert from PNG image format to JPEG image format

    Note: 
    - The 'targetDirectory' must exist already
    - Any metadata (e.g. XMP) that might be present in the source files
        is generally lost in the target files

    Parameters
    ----------
    sourceDirectory : str
        path to source directory containing images saved in PNG format. This method
        will attempt to load any file with the *.png extension, and it ignores other files

    targetDirectory : str
        target directory for writing the JPEG files

    """

    files: List[str] = os.listdir(sourceDirectory)
    print("png_directory_to_jpg() : found", len(files), "files in directory", sourceDirectory)

    for filename in files:

        # check file extension
        if len(filename) < 3:
            continue
        if filename[-3:] != "png":
            continue
        
        targetFilename = filename[0:-3] + "jpg"
        print("png_directory_to_jpg() :", filename, "->", targetFilename)

        image = cv2.imread(sourceDirectory + "/" + filename)    # read PNG image
        cv2.imwrite(targetDirectory + "/" + targetFilename, image)              # write JPEG




In [ ]:
png_directory_to_jpg(IMAGEHUB+'/20201014_goalpictures_PGN', 
                        IMAGEHUB+'/20201014_goalpictures')

png_directory_to_jpg() : found 490 files in directory /content/drive/My Drive/Robotsports/Robotsports_imagehub/20201014_goalpictures_PGN
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_05_color.png -> 3dsense_2020_10_14__21_56_05_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_18_color.png -> 3dsense_2020_10_14__21_56_18_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_20_color.png -> 3dsense_2020_10_14__21_56_20_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_22_color.png -> 3dsense_2020_10_14__21_56_22_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_24_color.png -> 3dsense_2020_10_14__21_56_24_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_27_color.png -> 3dsense_2020_10_14__21_56_27_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_29_color.png -> 3dsense_2020_10_14__21_56_29_color.jpg
png_directory_to_jpg() : 3dsense_2020_10_14__21_56_31_color.png -> 3dsense_2020_10_14__21_56_31_color.jpg
png_directory_t